# Step 1:

<font size = '5'>The code starts by importing necessary libraries for data manipulation, visualization, and machine learning. It then loads three datasets: a sample submission file (for Kaggle competition format), a test dataset, and a training dataset.</font>

In [ ]:
# Import standard libraries (pandas, mathplotlib, seaborn)
# Import specific tools from libraries: sklearn (score validation & tools, regression models)
# xgboost (gradient boosting); scipy(parameters for iteration)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import numpy as np
import math
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from scipy.stats import randint, uniform

# Load datasets
csv_file = "/kaggle/input/ai-x-data-supervised-pillar-spring-2025/sample_submission.csv"
df = pd.read_csv(csv_file)
print(df.head())

test_df = pd.read_csv("/kaggle/input/ai-x-data-supervised-pillar-spring-2025/test.csv")
print(dtest.head())

train_df = pd.read_csv("/kaggle/input/ai-x-data-supervised-pillar-spring-2025/train.csv")

# Step 2:

<font size='5'>This section handles missing values in both test and training datasets. For numeric columns, missing values are replaced with the median. For categorical columns (object type), missing values are replaced with the mode (most frequent value). Later in the code, a more robust approach is taken by first converting all columns to numeric where possible and then filling missing values.</font>

In [ ]:
# Forces numerical values for all non objects, converts objects to NaN values
train_df = train_df.apply(pd.to_numeric, errors='coerce')
test_df = test_df.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with medium
train_df.fillna(train_df.median(numeric_only=True), inplace=True)
test_df.fillna(test_df.median(numeric_only=True), inplace=True)

# Step 3:
<font size = '5'>This section visualizes the data for use to understand and see the relationships between variables. It creates scatter plots to see how average disbursed loans vary by school and school type, boxplots to compare loan distributions across school types, and calculates summary statistics for each school type. A histogram also shows the overall distribution of loan amounts.<font>

In [ ]:
# Scatter plot: Avg disbursed loan by school
plt.figure(figsize=(12, 6))
sns.scatterplot(x=dtrain["school"], y=dtrain["avg_disbursed_loan"], hue=dtrain["school_type"])
plt.title("Average Disbursed Loan by School")
plt.xlabel("School")
plt.ylabel("Avg Disbursed Loan")
plt.xticks(rotation=90)
plt.show()

#Boxplot the features and show the avg disbursed loan by school type
plt.figure(figsize=(10, 6))
sns.boxplot(x=dtrain["school_type"], y=dtrain["avg_disbursed_loan"])
plt.title("Average Disbursed Loan by School Type")
plt.xlabel("School Type")
plt.ylabel("Avg Disbursed Loan")
plt.show()

# Summary statistics for avg disbursed loan by school type
summary_stats = dtrain.groupby("school_type")["avg_disbursed_loan"].describe()
print("Summary Statistics by School Type:\n", summary_stats)

# Histogram of loan distribution
sns.histplot(train_df["avg_disbursed_loan"], bins=30, kde=True)
plt.title("Loan Distribution")
plt.show()

# Step 4: Feature Creation

<font size = '5'>This function creates new features from existing ones to potentially improve the model's performance. It creates combinations of tuition and career pay estimates, calculates the ratio of tuition to household income, and combines room/board price with enrollment. The code then prepares the feature matrix (X) and target variable (y) for model training. </font>


In [ ]:
#Function for making new features, when we call to it
def engineer_features(df):
    #Define list of columns that need to be numeric
    numeric_cols = ["in_state_tuition", "early_career_pay_estimate", "mid_career_pay_estimate",
                    "zip_median_household_income", "room_and_board_price", "total_enrollment"]

    #Make sure the columns are numeric and make sure they are numeric
    for col in numeric_cols:
        if col in df.columns:
            #Convert all the columns to numeric ones, if not make it none
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Allow new features to be made from what we currently have in our columns
    #For example, if we have in_state_tuition and early_career_pay_estimate then we can make tuition_earlyCareer
    if all(col in df.columns for col in ["in_state_tuition", "early_career_pay_estimate"]):
        df["tuition_earlyCareer"] = df["in_state_tuition"] * df["early_career_pay_estimate"]
    if all(col in df.columns for col in ["in_state_tuition", "mid_career_pay_estimate"]):
        df["tuition_midCareer"] = df["in_state_tuition"] * df["mid_career_pay_estimate"]
    if all(col in df.columns for col in ["in_state_tuition", "zip_median_household_income"]):
        df["tuition_income_ratio"] = df["in_state_tuition"] / (df["zip_median_household_income"] + 1)
    if all(col in df.columns for col in ["room_and_board_price", "total_enrollment"]):
        df["room_board_enrollment"] = df["room_and_board_price"] * df["total_enrollment"]

    #Now we return the dataframe with the new features that we made
    return df

# Prepare the features and the target from the training data
X = train_df.drop(["avg_disbursed_loan", "id"], axis=1, errors="ignore")
y = train_df["avg_disbursed_loan"].values

# Apply our function to make new features for the training set
X = engineer_features(X)
#Prepare the test set and drop ID
X_test = test_df.drop(["id"], axis=1, errors="ignore")
#This is the features engineered for the test set
X_test = engineer_features(X_test)

# Step 5: Model Training and Hyperparameter Tuning

<font size = '5'>This segment splits the data into training and validation sets, creates a custom MSE (Root Mean Squared Error) scoring function, and performs hyperparameter tuning for an XGBoost regression model using RandomizedSearchCV. The hyperparameter search explores different tree depths, learning rates, and sampling strategies to find the best combination.</font>

In [ ]:
# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Critical fix: Reset indexes after split and 
X_train.reset_index(drop=True, inplace=True)
X_val.reset_index(drop=True, inplace=True)
y_train = pd.Series(y_train).reset_index(drop=True)
y_val = pd.Series(y_val).reset_index(drop=True)

# MSE Scorer
def mse_metric(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# RMSE Scorer
def rmse_metric(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))
    
mse_scorer = make_scorer(mse_metric, greater_is_better=False)
rmse_scorer = make_scorer(rmse_metric, greater_is_better=False)

# XGBoost + Randomized Search
xgb_model = XGBRegressor(random_state=42, tree_method="hist")

param_distributions = {
    "n_estimators": randint(100, 300),
    "max_depth": randint(3, 7),
    "learning_rate": uniform(0.05, 0.15),
    "subsample": uniform(0.7, 0.3),
    "colsample_bytree": uniform(0.7, 0.3)
}

cv = KFold(n_splits=3, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_distributions,
    n_iter=10,
    scoring=rmse_scorer,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

#We do not want to overfit the model so we are setting paramters for early stopping by using an evaluation set and how many rounds if it does not improve
fit_params = {
    "eval_set": [(X_val, y_val)],
    "early_stopping_rounds": 30,
    "verbose": False
}


#FineTuning
#Fit model using RandSearchCV which samples the parameters we have set before
random_search.fit(X_train, y_train, **fit_params)

#Find the best parameter using our search 
#Give us the best model that we had with the different parameters according to our RMSE
print("\nBest Params:", random_search.best_params_)
#Hold the best model 
best_xgb = random_search.best_estimator_

# Step 6: Model Evaluation

<font size = '5'>The best XGBoost model from the hyperparameter search is evaluated on the validation set. Two metrics are reported: RMSE (lower is better), MSE (lower is better), and R² (higher is better). RMSE measures the average prediction error in the original units, while R² indicates the proportion of variance explained by the model.</font>

In [ ]:
# Validation
y_val_pred = best_xgb.predict(X_val) #random search funtion from XGB
#20% of original data for testing

val_rmse = rmse_metric(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"\nValidation RMSE: {val_rmse:.3f}")
print(f"Validation R^2: {val_r2:.3f}")

# Step 7: Feature Analysis

<font size =5>This section analyzes the importance of each feature in the trained XGBoost model. Features with very low importance (< 0.0001) are identified and optionally removed from the dataset to simplify the model.</font>


In [ ]:
# Feature importances
importances = best_xgb.feature_importances_
imp_series = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
print("\nTop 20 Features by Importance:")
print(imp_series.head(20))

#best_xgb.feature_importances_ retrieves the importance scores
#of each feature.
#These scores indicate how much each feature contributes 
#to the model’s predictions.
#Higher values = more important features; lower values = less useful features.

# Optional: drop low-importance features
low_importance = imp_series[imp_series < 1e-4].index
if len(low_importance) > 0:
    print("\nDropping these near-zero-importance features:", list(low_importance))
    X_train = X_train.drop(columns=low_importance, errors="ignore")
    X_val = X_val.drop(columns=low_importance, errors="ignore")
    X_test = X_test.drop(columns=low_importance, errors="ignore")

# Step 8: Final Model Training and Prediction

<font size = '5'>This final section combines the training and validation sets to train the model on all available data. It also creates an ensemble model by averaging predictions from XGBoost and Random Forest models, which often yields better performance than either model alone. The ensemble is evaluated on the validation set, and then used to generate predictions for the test set. Finally, a submission file is created for the Kaggle competition.</font>


In [ ]:
# Retrain on all data
X_train.reset_index(drop=True, inplace=True)
X_val.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)

X_combined = pd.concat([X_train, X_val], ignore_index=True)
y_combined = pd.concat([y_train, y_val], ignore_index=True)

best_xgb.fit(
    X_combined, 
    y_combined,
    early_stopping_rounds=30,
    eval_set=[(X_combined, y_combined)],
    verbose=False
)

# Optional ensemble
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_combined, y_combined)

y_val_pred_xgb = best_xgb.predict(X_val)
y_val_pred_rf = rf.predict(X_val)
y_val_pred_ensemble = 0.5 * y_val_pred_xgb + 0.5 * y_val_pred_rf

ensemble_rmse = rmse_metric(y_val, y_val_pred_ensemble)
#measures the average magnitude of errors 
#(how far predictions are from actual values) 
#in the same unit as the target variable.
ensemble_r2 = r2_score(y_val, y_val_pred_ensemble)

print(f"\nEnsemble RMSE: {ensemble_rmse:.3f}")
print(f"Ensemble R^2: {ensemble_r2:.3f}")

# Final predictions
y_test_pred_xgb = best_xgb.predict(X_test)
y_test_pred_rf = rf.predict(X_test)
y_test_pred_ensemble = 0.5 * y_test_pred_xgb + 0.5 * y_test_pred_rf

submission = pd.DataFrame({
    "id": test_df["id"],
    "avg_disbursed_loan": y_test_pred_ensemble
})
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")
print(submission)